In [1]:
from google.colab import drive
import sys
import pandas as pd
import numpy as np

# **Load Data**

In [2]:
# Mount Google Drive
drive.mount('/content/drive')

# Get the absolute path of the current folder
abspath_curr = '/content/drive/My Drive/'

Mounted at /content/drive


In [3]:
transactions=pd.read_csv(abspath_curr+'transactions.csv',index_col=[0])
transactions

,accountNumber,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,posConditionCode,merchantCategoryCode,currentExpDate,accountOpenDate,dateOfLastAddressChange,cardCVV,enteredCVV,cardLast4Digits,transactionType,echoBuffer,currentBalance,merchantCity,merchantState,merchantZip,cardPresent,posOnPremises,recurringAuthInd,expirationDateKeyInMatch,isFraud
0,737265056,737265056,5000.0,5000.00,2016-08-13T14:27:32,98.55,Uber,US,US,2.0,1.0,rideshare,06/2023,2015-03-14,2015-03-14,414,414,1803,PURCHASE,NaN,0.00,NaN,NaN,NaN,False,NaN,NaN,False,False
1,737265056,737265056,5000.0,5000.00,2016-10-11T05:05:54,74.51,AMC #191138,US,US,9.0,1.0,entertainment,02/2024,2015-03-14,2015-03-14,486,486,767,PURCHASE,NaN,0.00,NaN,NaN,NaN,True,NaN,NaN,False,False
2,737265056,737265056,5000.0,5000.00,2016-11-08T09:18:39,7.47,Play Store,US,US,9.0,1.0,mobileapps,08/2025,2015-03-14,2015-03-14,486,486,767,PURCHASE,NaN,0.00,NaN,NaN,NaN,False,NaN,NaN,False,False
3,737265056,737265056,5000.0,5000.00,2016-12-10T02:14:50,7.47,Play Store,US,US,9.0,1.0,mobileapps,08/2025,2015-03-14,2015-03-14,486,486,767,PURCHASE,NaN,0.00,NaN,NaN,NaN,False,NaN,NaN,False,False
4,830329091,830329091,5000.0,5000.00,2016-03-24T21:04:46,71.18,Tim Hortons #947751,US,US,2.0,1.0,fastfood,10/2029,2015-08-06,2015-08-06,885,885,3143,PURCHASE,NaN,0.00,NaN,NaN,NaN,True,NaN,NaN,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786358,732852505,732852505,50000.0,48904.96,2016-12-22T18:44:12,119.92,Lyft,US,US,90.0,1.0,rideshare,12/2022,2012-08-23,2012-08-23,936,936,3783,PURCHASE,NaN,1095.04,NaN,NaN,NaN,False,NaN,NaN,False,False
786359,732852505,732852505,50000.0,48785.04,2016-12-25T16:20:34,18.89,hulu.com,US,US,9.0,1.0,online_subscriptions,08/2023,2012-08-23,2012-08-23,939,939,3388,PURCHASE,NaN,1214.96,NaN,NaN,NaN,False,NaN,NaN,False,False
786360,732852505,732852505,50000.0,48766.15,2016-12-27T15:46:24,49.43,Lyft,US,US,2.0,1.0,rideshare,08/2025,2012-08-23,2012-08-23,936,936,3783,PURCHASE,NaN,1233.85,NaN,NaN,NaN,False,NaN,NaN,False,False
786361,732852505,732852505,50000.0,48716.72,2016-12-29T00:30:55,49.89,walmart.com,US,US,9.0,99.0,online_retail,07/2022,2012-08-23,2012-08-23,939,939,3388,PURCHASE,NaN,1283.28,NaN,NaN,NaN,False,NaN,NaN,False,False


# **Data Wrangling**

##Set the transaction key

In [4]:
# Set the transaction key
def concat(x):
    y=str('T') + str(x)
    return y

transactions.reset_index(drop=False,inplace=True)
transactions['transactionKey']=transactions['index'].apply(lambda x: concat(x))
transactions.drop(['index'],axis=1,inplace=True)

##**Reversed Transactions**


Assumptions

1.  I assume the reversed transactions contain both purchase and reversal transaction types.
2.  I assume the reversed transactions have the same transaction amounts beween two transaction types.



In [5]:
# Find the transactions with "Purchase" and "Reversal" 
transa=transactions[(transactions['transactionType']== 'PURCHASE')|(transactions['transactionType']=='REVERSAL')][['customerId','merchantName','transactionAmount','transactionType','transactionKey']]
transa

,customerId,merchantName,transactionAmount,transactionType,transactionKey
0,737265056,Uber,98.55,PURCHASE,T0
1,737265056,AMC #191138,74.51,PURCHASE,T1
2,737265056,Play Store,7.47,PURCHASE,T2
3,737265056,Play Store,7.47,PURCHASE,T3
4,830329091,Tim Hortons #947751,71.18,PURCHASE,T4
...,...,...,...,...,...
786358,732852505,Lyft,119.92,PURCHASE,T786358
786359,732852505,hulu.com,18.89,PURCHASE,T786359
786360,732852505,Lyft,49.43,PURCHASE,T786360
786361,732852505,walmart.com,49.89,PURCHASE,T786361


In [7]:
#Group data by 'customerId' and 'merchantName' 
#Select transactions in both same customer and merchant
count_ = transactions.groupby(['customerId','merchantName']).count()
count_Amount = count_[count_['transactionAmount']>1].reset_index()[['customerId','merchantName']]
count_Amount

,customerId,merchantName
0,100088067,AMC #552863
1,100088067,AMC #724446
2,100088067,AMC #79863
3,100088067,Apple iTunes
4,100088067,Blue Mountain Online Services
...,...,...
79848,999283629,sears.com
79849,999283629,staples.com
79850,999283629,target.com
79851,999283629,walmart.com


In [9]:
# Combine the data 
records = pd.merge(count_Amount, transa, left_on=['customerId','merchantName'],right_on=['customerId','merchantName'])
records 

,customerId,merchantName,transactionAmount,transactionType,transactionKey
0,100088067,AMC #552863,485.92,PURCHASE,T541951
1,100088067,AMC #552863,53.45,PURCHASE,T541958
2,100088067,AMC #724446,71.35,PURCHASE,T541909
3,100088067,AMC #724446,55.04,PURCHASE,T541946
4,100088067,AMC #724446,249.19,PURCHASE,T541954
...,...,...,...,...,...
640240,999389635,Play Store,4.62,PURCHASE,T108110
640241,999389635,Play Store,4.62,PURCHASE,T108112
640242,999389635,Play Store,4.62,PURCHASE,T108113
640243,999389635,Play Store,4.62,PURCHASE,T108114


In [11]:
# Group data by the same transaction amounts
# Drop rows with only one transaction
count_records = records.groupby(['customerId','merchantName','transactionAmount']).count()
count_data = count_records[count_records['transactionType']>1].reset_index()[['customerId','merchantName','transactionAmount']]
count_data

,customerId,merchantName,transactionAmount
0,100088067,AMC #79863,22.32
1,100088067,Apple iTunes,3.96
2,100088067,Fresh Flowers,411.35
3,100088067,Shell Gas #256420,60.41
4,100108752,Shell Gas #494785,45.35
...,...,...,...
32438,999283629,Franks Sandwitch Bar,150.58
32439,999283629,Lyft,96.56
32440,999283629,abc.com,13.04
32441,999283629,apple.com,407.23


In [12]:
# Map the data between count_data and records
card_data = pd.merge(count_data, records, left_on=['customerId','merchantName','transactionAmount'],right_on=['customerId','merchantName','transactionAmount'])
card_data.drop_duplicates(subset=['customerId','merchantName','transactionAmount','transactionType'],keep='first',inplace=True)
card_data

,customerId,merchantName,transactionAmount,transactionType,transactionKey
0,100088067,AMC #79863,22.32,PURCHASE,T541963
1,100088067,AMC #79863,22.32,REVERSAL,T541964
2,100088067,Apple iTunes,3.96,PURCHASE,T541919
10,100088067,Fresh Flowers,411.35,PURCHASE,T541955
12,100088067,Shell Gas #256420,60.41,PURCHASE,T541953
...,...,...,...,...,...
101049,999283629,Lyft,96.56,REVERSAL,T670052
101050,999283629,abc.com,13.04,PURCHASE,T670063
101056,999283629,apple.com,407.23,PURCHASE,T670084
101057,999283629,apple.com,407.23,REVERSAL,T670091


In [13]:
reversal_data = card_data.groupby(['customerId','merchantName','transactionAmount']).count()
reversal_ = reversal_data[reversal_data['transactionType']>1].reset_index()[['customerId','merchantName','transactionAmount','transactionKey']]
reversal_.drop(columns=['transactionKey'],inplace=True)
reversal_


,customerId,merchantName,transactionAmount
0,100088067,AMC #79863,22.32
1,100328049,Lyft,43.74
2,100328049,McDonalds #423357,284.97
3,100737756,34th BBQ #436606,93.67
4,100737756,Best Bistro #262998,501.29
...,...,...,...
17766,999258704,gap.com,72.93
17767,999283629,Delta Airlines,247.40
17768,999283629,Franks Sandwitch Bar,150.58
17769,999283629,Lyft,96.56


###Identify reversed transactions





In [14]:
# Get the reversal transactions
reversal_transactions = pd.merge(reversal_, card_data, left_on = ['customerId','merchantName','transactionAmount'],right_on=['customerId','merchantName','transactionAmount'])
reversal_transactions.drop_duplicates(subset=['customerId','merchantName','transactionAmount','transactionType'],keep='first',inplace=True)
reversal_transactions

,customerId,merchantName,transactionAmount,transactionType,transactionKey
0,100088067,AMC #79863,22.32,PURCHASE,T541963
1,100088067,AMC #79863,22.32,REVERSAL,T541964
2,100328049,Lyft,43.74,PURCHASE,T87919
3,100328049,Lyft,43.74,REVERSAL,T87920
4,100328049,McDonalds #423357,284.97,PURCHASE,T87935
...,...,...,...,...,...
35537,999283629,Franks Sandwitch Bar,150.58,REVERSAL,T670109
35538,999283629,Lyft,96.56,PURCHASE,T670051
35539,999283629,Lyft,96.56,REVERSAL,T670052
35540,999283629,apple.com,407.23,PURCHASE,T670084




> **total number of transactions for the reversed transactions is 17771**



In [15]:
reversal_transactions['transactionType'].value_counts()

PURCHASE    17771
REVERSAL    17771
Name: transactionType, dtype: int64



>   **total dollar amount for the reversed transactions corresponds to $2667031.44**



In [16]:
reversal_transactions.groupby(['transactionType']).sum()['transactionAmount']

transactionType
PURCHASE    2667031.44
REVERSAL    2667031.44
Name: transactionAmount, dtype: float64

In [17]:
transactions['transactionType'].value_counts()

PURCHASE                745193
REVERSAL                 20303
ADDRESS_VERIFICATION     20169
Name: transactionType, dtype: int64

* There are 2532 reversal transactions which I can not identify.

* I think there are two reasons: 1)The amounts in purchase and reversal can be different.2) Data may be incomplete.





 
 

In [18]:
transactions.groupby(['transactionType']).sum()['transactionAmount']

transactionType
ADDRESS_VERIFICATION    0.000000e+00
PURCHASE                1.047903e+08
REVERSAL                2.821792e+06
Name: transactionAmount, dtype: float64



> ** Total dollar amount for the reversal transactions which I can not identified corresponds to $154450.11.**



In [20]:
total_=transa[(transa['transactionType']=='REVERSAL')][['customerId','merchantName','transactionAmount','transactionType']]
reseveral_su=reversal_transactions[(reversal_transactions['transactionType']=='REVERSAL')]
reserversal_left=reseveral_su.append(total_)
reserversal_left.drop_duplicates(keep=False,inplace=True)
reserversal_left.reset_index()
reserversal_left.groupby(['transactionType']).sum()['transactionAmount']

transactionType
REVERSAL    154450.11
Name: transactionAmount, dtype: float64



###Delete the reversed transactions



In [19]:
credit_card_no_reversed=transactions[~(transactions.transactionKey.isin(reversal_transactions.transactionKey))]
credit_card_no_reversed

,accountNumber,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,posConditionCode,merchantCategoryCode,currentExpDate,accountOpenDate,dateOfLastAddressChange,cardCVV,enteredCVV,cardLast4Digits,transactionType,echoBuffer,currentBalance,merchantCity,merchantState,merchantZip,cardPresent,posOnPremises,recurringAuthInd,expirationDateKeyInMatch,isFraud,transactionKey
0,737265056,737265056,5000.0,5000.00,2016-08-13T14:27:32,98.55,Uber,US,US,2.0,1.0,rideshare,06/2023,2015-03-14,2015-03-14,414,414,1803,PURCHASE,NaN,0.00,NaN,NaN,NaN,False,NaN,NaN,False,False,T0
1,737265056,737265056,5000.0,5000.00,2016-10-11T05:05:54,74.51,AMC #191138,US,US,9.0,1.0,entertainment,02/2024,2015-03-14,2015-03-14,486,486,767,PURCHASE,NaN,0.00,NaN,NaN,NaN,True,NaN,NaN,False,False,T1
2,737265056,737265056,5000.0,5000.00,2016-11-08T09:18:39,7.47,Play Store,US,US,9.0,1.0,mobileapps,08/2025,2015-03-14,2015-03-14,486,486,767,PURCHASE,NaN,0.00,NaN,NaN,NaN,False,NaN,NaN,False,False,T2
3,737265056,737265056,5000.0,5000.00,2016-12-10T02:14:50,7.47,Play Store,US,US,9.0,1.0,mobileapps,08/2025,2015-03-14,2015-03-14,486,486,767,PURCHASE,NaN,0.00,NaN,NaN,NaN,False,NaN,NaN,False,False,T3
4,830329091,830329091,5000.0,5000.00,2016-03-24T21:04:46,71.18,Tim Hortons #947751,US,US,2.0,1.0,fastfood,10/2029,2015-08-06,2015-08-06,885,885,3143,PURCHASE,NaN,0.00,NaN,NaN,NaN,True,NaN,NaN,False,False,T4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786358,732852505,732852505,50000.0,48904.96,2016-12-22T18:44:12,119.92,Lyft,US,US,90.0,1.0,rideshare,12/2022,2012-08-23,2012-08-23,936,936,3783,PURCHASE,NaN,1095.04,NaN,NaN,NaN,False,NaN,NaN,False,False,T786358
786359,732852505,732852505,50000.0,48785.04,2016-12-25T16:20:34,18.89,hulu.com,US,US,9.0,1.0,online_subscriptions,08/2023,2012-08-23,2012-08-23,939,939,3388,PURCHASE,NaN,1214.96,NaN,NaN,NaN,False,NaN,NaN,False,False,T786359
786360,732852505,732852505,50000.0,48766.15,2016-12-27T15:46:24,49.43,Lyft,US,US,2.0,1.0,rideshare,08/2025,2012-08-23,2012-08-23,936,936,3783,PURCHASE,NaN,1233.85,NaN,NaN,NaN,False,NaN,NaN,False,False,T786360
786361,732852505,732852505,50000.0,48716.72,2016-12-29T00:30:55,49.89,walmart.com,US,US,9.0,99.0,online_retail,07/2022,2012-08-23,2012-08-23,939,939,3388,PURCHASE,NaN,1283.28,NaN,NaN,NaN,False,NaN,NaN,False,False,T786361


##**Multi-Wipe Transactions**


Assumption:

 I assume the multi-wipe transactions happened at the same day.




In [20]:
Multi_wipe_data = credit_card_no_reversed[(credit_card_no_reversed['transactionType'] == 'PURCHASE')][['customerId','transactionAmount','merchantName','transactionType','transactionDateTime','transactionKey']]
Multi_wipe_data['customerId'] = Multi_wipe_data['customerId'].astype(dtype='int64')
Multi_wipe_data['transaction_DateTime'] = Multi_wipe_data['transactionDateTime'].apply(lambda x:x.split('T')[0])
Multi_wipe_data.drop(labels='transactionDateTime',axis=1,inplace=True)
Multi_wipe_data

,customerId,transactionAmount,merchantName,transactionType,transactionKey,transaction_DateTime
0,737265056,98.55,Uber,PURCHASE,T0,2016-08-13
1,737265056,74.51,AMC #191138,PURCHASE,T1,2016-10-11
2,737265056,7.47,Play Store,PURCHASE,T2,2016-11-08
3,737265056,7.47,Play Store,PURCHASE,T3,2016-12-10
4,830329091,71.18,Tim Hortons #947751,PURCHASE,T4,2016-03-24
...,...,...,...,...,...,...
786358,732852505,119.92,Lyft,PURCHASE,T786358,2016-12-22
786359,732852505,18.89,hulu.com,PURCHASE,T786359,2016-12-25
786360,732852505,49.43,Lyft,PURCHASE,T786360,2016-12-27
786361,732852505,49.89,walmart.com,PURCHASE,T786361,2016-12-29


In [21]:
# Group data by the same date, same customer and same merchant
mul_count = Multi_wipe_data.groupby(['customerId','merchantName','transaction_DateTime']).count()
mul_count = mul_count.reset_index()

In [22]:
# Select the transactions 
mutiple_counts = mul_count[mul_count['transactionAmount']>1][['customerId','merchantName','transaction_DateTime','transactionKey']]
mutiple_counts.drop(columns=['transactionKey'],inplace=True)
mutiple_counts

,customerId,merchantName,transaction_DateTime
42,100088067,Fresh Flowers,2016-10-16
229,100737756,1st Sandwitch Bar #396252,2016-05-23
245,100737756,34th BBQ #166379,2016-07-10
308,100737756,Franks Deli,2016-01-18
362,100737756,South Steakhouse #73819,2016-07-02
...,...,...,...
658411,999257059,Oklahoma City Times,2016-03-14
658452,999257059,Wall Street News,2016-01-14
658465,999257059,Wall Street Times,2016-02-25
658473,999257059,Wall Street Times,2016-12-17


###Identify multi-wipe transactions

In [23]:
Multiwipe=pd.merge(mutiple_counts,Multi_wipe_data,left_on=['customerId','merchantName','transaction_DateTime'], right_on=['customerId','merchantName','transaction_DateTime'])
Multiwipe

,customerId,merchantName,transaction_DateTime,transactionAmount,transactionType,transactionKey
0,100088067,Fresh Flowers,2016-10-16,411.35,PURCHASE,T541955
1,100088067,Fresh Flowers,2016-10-16,411.35,PURCHASE,T541956
2,100737756,1st Sandwitch Bar #396252,2016-05-23,61.02,PURCHASE,T151492
3,100737756,1st Sandwitch Bar #396252,2016-05-23,121.85,PURCHASE,T151493
4,100737756,34th BBQ #166379,2016-07-10,43.25,PURCHASE,T151517
...,...,...,...,...,...,...
105333,999257059,Wall Street Times,2016-02-25,172.02,PURCHASE,T569679
105334,999257059,Wall Street Times,2016-12-17,5.40,PURCHASE,T569799
105335,999257059,Wall Street Times,2016-12-17,390.10,PURCHASE,T569800
105336,999283629,alibaba.com,2016-12-07,582.39,PURCHASE,T670115


In [24]:
# Get the normal transactions in Multi-wipe transactions
Multiwipe_Normal=Multiwipe.drop_duplicates(subset=['customerId','merchantName','transaction_DateTime'],keep='first')
Multiwipe_Normal

,customerId,merchantName,transaction_DateTime,transactionAmount,transactionType,transactionKey
0,100088067,Fresh Flowers,2016-10-16,411.35,PURCHASE,T541955
2,100737756,1st Sandwitch Bar #396252,2016-05-23,61.02,PURCHASE,T151492
4,100737756,34th BBQ #166379,2016-07-10,43.25,PURCHASE,T151517
6,100737756,Franks Deli,2016-01-18,693.50,PURCHASE,T151426
9,100737756,South Steakhouse #73819,2016-07-02,211.22,PURCHASE,T151512
...,...,...,...,...,...,...
105328,999257059,Oklahoma City Times,2016-03-14,62.14,PURCHASE,T569684
105330,999257059,Wall Street News,2016-01-14,47.58,PURCHASE,T569653
105332,999257059,Wall Street Times,2016-02-25,4.47,PURCHASE,T569678
105334,999257059,Wall Street Times,2016-12-17,5.40,PURCHASE,T569799


**Total number of transactions for the multi-swipe transactions is 68698.**

In [88]:
105338-36640

68698

In [25]:
Multiwipe_Du=pd.merge(Multiwipe_Normal,Multiwipe,on=['customerId','merchantName','transaction_DateTime','transactionKey','transactionAmount','transactionType'],how='right',indicator=True)
Multiwipe_Du=Multiwipe_Du[(Multiwipe_Du['_merge']== 'right_only')][['customerId','merchantName','transaction_DateTime','transactionKey','transactionAmount','transactionType']]
Multiwipe_Du

,customerId,merchantName,transaction_DateTime,transactionKey,transactionAmount,transactionType
1,100088067,Fresh Flowers,2016-10-16,T541956,411.35,PURCHASE
3,100737756,1st Sandwitch Bar #396252,2016-05-23,T151493,121.85,PURCHASE
5,100737756,34th BBQ #166379,2016-07-10,T151518,43.25,PURCHASE
7,100737756,Franks Deli,2016-01-18,T151427,693.50,PURCHASE
8,100737756,Franks Deli,2016-01-18,T151428,693.50,PURCHASE
...,...,...,...,...,...,...
105329,999257059,Oklahoma City Times,2016-03-14,T569685,242.88,PURCHASE
105331,999257059,Wall Street News,2016-01-14,T569654,47.58,PURCHASE
105333,999257059,Wall Street Times,2016-02-25,T569679,172.02,PURCHASE
105335,999257059,Wall Street Times,2016-12-17,T569800,390.10,PURCHASE


**Total dollar amount for the multi-swipe transactions is $10268522.62.**

In [26]:
Multiwipe_Du.sum()['transactionAmount']

10268522.62

###Delete the multi-wipe transactions

In [29]:
credit_card_no_duplicate=credit_card_no_reversed[~(credit_card_no_reversed.transactionKey.isin(Multiwipe_Du.transactionKey))]
credit_card_no_duplicate

,accountNumber,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,posConditionCode,merchantCategoryCode,currentExpDate,accountOpenDate,dateOfLastAddressChange,cardCVV,enteredCVV,cardLast4Digits,transactionType,echoBuffer,currentBalance,merchantCity,merchantState,merchantZip,cardPresent,posOnPremises,recurringAuthInd,expirationDateKeyInMatch,isFraud,transactionKey
0,737265056,737265056,5000.0,5000.00,2016-08-13T14:27:32,98.55,Uber,US,US,2.0,1.0,rideshare,06/2023,2015-03-14,2015-03-14,414,414,1803,PURCHASE,NaN,0.00,NaN,NaN,NaN,False,NaN,NaN,False,False,T0
1,737265056,737265056,5000.0,5000.00,2016-10-11T05:05:54,74.51,AMC #191138,US,US,9.0,1.0,entertainment,02/2024,2015-03-14,2015-03-14,486,486,767,PURCHASE,NaN,0.00,NaN,NaN,NaN,True,NaN,NaN,False,False,T1
2,737265056,737265056,5000.0,5000.00,2016-11-08T09:18:39,7.47,Play Store,US,US,9.0,1.0,mobileapps,08/2025,2015-03-14,2015-03-14,486,486,767,PURCHASE,NaN,0.00,NaN,NaN,NaN,False,NaN,NaN,False,False,T2
3,737265056,737265056,5000.0,5000.00,2016-12-10T02:14:50,7.47,Play Store,US,US,9.0,1.0,mobileapps,08/2025,2015-03-14,2015-03-14,486,486,767,PURCHASE,NaN,0.00,NaN,NaN,NaN,False,NaN,NaN,False,False,T3
4,830329091,830329091,5000.0,5000.00,2016-03-24T21:04:46,71.18,Tim Hortons #947751,US,US,2.0,1.0,fastfood,10/2029,2015-08-06,2015-08-06,885,885,3143,PURCHASE,NaN,0.00,NaN,NaN,NaN,True,NaN,NaN,False,False,T4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786358,732852505,732852505,50000.0,48904.96,2016-12-22T18:44:12,119.92,Lyft,US,US,90.0,1.0,rideshare,12/2022,2012-08-23,2012-08-23,936,936,3783,PURCHASE,NaN,1095.04,NaN,NaN,NaN,False,NaN,NaN,False,False,T786358
786359,732852505,732852505,50000.0,48785.04,2016-12-25T16:20:34,18.89,hulu.com,US,US,9.0,1.0,online_subscriptions,08/2023,2012-08-23,2012-08-23,939,939,3388,PURCHASE,NaN,1214.96,NaN,NaN,NaN,False,NaN,NaN,False,False,T786359
786360,732852505,732852505,50000.0,48766.15,2016-12-27T15:46:24,49.43,Lyft,US,US,2.0,1.0,rideshare,08/2025,2012-08-23,2012-08-23,936,936,3783,PURCHASE,NaN,1233.85,NaN,NaN,NaN,False,NaN,NaN,False,False,T786360
786361,732852505,732852505,50000.0,48716.72,2016-12-29T00:30:55,49.89,walmart.com,US,US,9.0,99.0,online_retail,07/2022,2012-08-23,2012-08-23,939,939,3388,PURCHASE,NaN,1283.28,NaN,NaN,NaN,False,NaN,NaN,False,False,T786361


In [30]:
786363-682123

104240


*   I fond the total number of duplicate transactions was 104,240.
*   There were lots of monthly subscriptions transactions.



# **Data Processing**

In [31]:
# Drop columns with NaN values
process_data=credit_card_no_duplicate.drop(columns=['transactionDateTime','echoBuffer','merchantCity','merchantState','merchantZip','posOnPremises','recurringAuthInd'])
process_data

,accountNumber,customerId,creditLimit,availableMoney,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,posConditionCode,merchantCategoryCode,currentExpDate,accountOpenDate,dateOfLastAddressChange,cardCVV,enteredCVV,cardLast4Digits,transactionType,currentBalance,cardPresent,expirationDateKeyInMatch,isFraud,transactionKey
0,737265056,737265056,5000.0,5000.00,98.55,Uber,US,US,2.0,1.0,rideshare,06/2023,2015-03-14,2015-03-14,414,414,1803,PURCHASE,0.00,False,False,False,T0
1,737265056,737265056,5000.0,5000.00,74.51,AMC #191138,US,US,9.0,1.0,entertainment,02/2024,2015-03-14,2015-03-14,486,486,767,PURCHASE,0.00,True,False,False,T1
2,737265056,737265056,5000.0,5000.00,7.47,Play Store,US,US,9.0,1.0,mobileapps,08/2025,2015-03-14,2015-03-14,486,486,767,PURCHASE,0.00,False,False,False,T2
3,737265056,737265056,5000.0,5000.00,7.47,Play Store,US,US,9.0,1.0,mobileapps,08/2025,2015-03-14,2015-03-14,486,486,767,PURCHASE,0.00,False,False,False,T3
4,830329091,830329091,5000.0,5000.00,71.18,Tim Hortons #947751,US,US,2.0,1.0,fastfood,10/2029,2015-08-06,2015-08-06,885,885,3143,PURCHASE,0.00,True,False,False,T4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786358,732852505,732852505,50000.0,48904.96,119.92,Lyft,US,US,90.0,1.0,rideshare,12/2022,2012-08-23,2012-08-23,936,936,3783,PURCHASE,1095.04,False,False,False,T786358
786359,732852505,732852505,50000.0,48785.04,18.89,hulu.com,US,US,9.0,1.0,online_subscriptions,08/2023,2012-08-23,2012-08-23,939,939,3388,PURCHASE,1214.96,False,False,False,T786359
786360,732852505,732852505,50000.0,48766.15,49.43,Lyft,US,US,2.0,1.0,rideshare,08/2025,2012-08-23,2012-08-23,936,936,3783,PURCHASE,1233.85,False,False,False,T786360
786361,732852505,732852505,50000.0,48716.72,49.89,walmart.com,US,US,9.0,99.0,online_retail,07/2022,2012-08-23,2012-08-23,939,939,3388,PURCHASE,1283.28,False,False,False,T786361


## **Sampling**

Since this dataset is highly imbalanced, I retain the fraud data and sample from true transactions.
 

In [32]:
process_data['isFraud'].value_counts()# Imbalanced Data

False    671906
True      10217
Name: isFraud, dtype: int64

In [33]:
processed=process_data.copy()

In [34]:
# Find customers who had fraud transations
fraud_data=processed[processed.isFraud==True]
fraud_data.customerId.nunique()
processed=processed.reset_index(drop=True)
customer_data=fraud_data.customerId.unique() 

In [53]:
# Find true transactions with same customers
customer_fraud=processed[processed.customerId.isin(customer_data)]
true_tr=customer_fraud[customer_fraud.isFraud==False] 
df_false_1=true_tr.sample(frac=0.02,random_state=42)
df_false_1=df_false_1.reset_index(drop=True)
df_false_1

,accountNumber,customerId,creditLimit,availableMoney,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,posConditionCode,merchantCategoryCode,currentExpDate,accountOpenDate,dateOfLastAddressChange,cardCVV,enteredCVV,cardLast4Digits,transactionType,currentBalance,cardPresent,expirationDateKeyInMatch,isFraud,transactionKey
0,358749735,358749735,15000.0,9992.97,19.70,KFC #42585,US,US,5.0,1.0,fastfood,09/2032,2014-12-31,2014-12-31,971,971,8366,PURCHASE,5007.03,True,False,False,T696646
1,699239145,699239145,15000.0,3935.64,193.96,Uber,US,US,9.0,1.0,rideshare,04/2020,2013-12-05,2013-12-05,351,351,1688,PURCHASE,11064.36,False,False,False,T514646
2,854042767,854042767,1000.0,111.98,130.56,Krispy Kreme #205879,US,US,2.0,1.0,fastfood,02/2029,2015-11-13,2016-11-19,554,554,5358,PURCHASE,888.02,True,False,False,T80703
3,770035360,770035360,2500.0,1793.41,46.06,sears.com,US,US,2.0,1.0,online_retail,12/2021,2014-07-08,2014-07-08,946,946,3166,PURCHASE,706.59,False,False,False,T687053
4,894938833,894938833,15000.0,4172.20,286.55,NYSC #160820,US,US,9.0,1.0,health,10/2024,2014-11-05,2016-10-04,689,689,9102,PURCHASE,10827.80,False,False,False,T215349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10850,259732658,259732658,50000.0,50000.00,177.19,Oklahoma City Times,US,US,5.0,1.0,subscriptions,01/2024,2014-08-05,2014-08-05,248,248,413,PURCHASE,0.00,False,False,False,T564827
10851,119937133,119937133,20000.0,19592.55,94.98,West End Beauty #72256,US,US,9.0,1.0,personal care,03/2032,2015-06-03,2015-06-03,123,123,5122,PURCHASE,407.45,True,False,False,T563928
10852,118367557,118367557,5000.0,2261.03,174.12,gap.com,US,US,9.0,8.0,online_retail,03/2030,2014-11-26,2014-11-26,252,252,5738,PURCHASE,2738.97,False,False,False,T450866
10853,338839426,338839426,250.0,161.89,8.08,hulu.com,US,US,2.0,1.0,online_subscriptions,08/2033,2015-04-09,2015-04-09,714,714,3599,PURCHASE,88.11,False,False,False,T109905


In [54]:
df_false_1.customerId.nunique()

1676

In [56]:
# Sample from other cusmtoers who did not have fraudulant transactions
true_data=processed[processed.isFraud==False]
df_new=true_data[~true_data.transactionKey.isin(true_tr.transactionKey)] # take transactions which 
df_new
df_false_2=df_new.sample(frac=0.08,random_state=42)
df_false_2=df_false_2.reset_index(drop=True)
df_false_2

,accountNumber,customerId,creditLimit,availableMoney,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,posConditionCode,merchantCategoryCode,currentExpDate,accountOpenDate,dateOfLastAddressChange,cardCVV,enteredCVV,cardLast4Digits,transactionType,currentBalance,cardPresent,expirationDateKeyInMatch,isFraud,transactionKey
0,167830614,167830614,2500.0,381.37,724.00,Dinosaur Restaurant,US,US,2.0,1.0,food,10/2029,2015-12-04,2015-12-04,191,191,9374,PURCHASE,2118.63,True,False,False,T594344
1,315850074,315850074,15000.0,10830.75,0.00,Regal Cinemas #05791,US,US,5.0,1.0,entertainment,06/2032,2015-08-31,2015-08-31,764,764,7731,ADDRESS_VERIFICATION,4169.25,False,False,False,T286603
2,144105109,144105109,15000.0,13007.27,39.56,Sunoco Gas #946319,US,US,2.0,1.0,fuel,12/2026,2013-01-05,2016-04-26,347,347,0,PURCHASE,1992.73,True,False,False,T545800
3,628652984,628652984,2500.0,1482.04,14.17,1st Sandwitch Bar #119707,US,US,9.0,1.0,food,08/2027,2013-04-03,2013-04-03,842,842,79,PURCHASE,1017.96,True,False,False,T780361
4,887139069,887139069,2500.0,2173.42,331.59,AMC #74121,US,US,5.0,1.0,entertainment,04/2025,2015-11-15,2015-11-15,382,382,0,PURCHASE,326.58,True,False,False,T372407
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10329,142462254,142462254,7500.0,6506.83,3.46,Auntie Anne's #580327,US,US,5.0,1.0,fastfood,12/2027,2015-09-06,2016-05-03,362,362,7831,PURCHASE,993.17,True,False,False,T160744
10330,673695283,673695283,250.0,53.78,167.73,Universe Massage #725790,US,US,5.0,1.0,personal care,07/2033,2013-01-05,2013-01-05,532,532,231,PURCHASE,196.22,True,False,False,T568623
10331,101394465,101394465,5000.0,4690.04,51.07,Uber,US,US,5.0,1.0,rideshare,09/2027,2014-09-04,2014-09-04,915,915,9176,PURCHASE,309.96,False,False,False,T724633
10332,858587070,858587070,10000.0,9811.00,198.79,Universe Massage #206797,US,US,5.0,1.0,personal care,12/2031,2015-03-20,2015-03-20,602,602,7371,PURCHASE,189.00,True,False,False,T434332


In [57]:
credit_card=pd.concat([df_false_1,df_false_2,fraud_data],axis=0) # merge dataframes vertically
credit_card.isFraud.value_counts()

False    21189
True     10217
Name: isFraud, dtype: int64

## **Feature Engineering**

In [58]:
df_tr=credit_card.copy()
df_tr

,accountNumber,customerId,creditLimit,availableMoney,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,posConditionCode,merchantCategoryCode,currentExpDate,accountOpenDate,dateOfLastAddressChange,cardCVV,enteredCVV,cardLast4Digits,transactionType,currentBalance,cardPresent,expirationDateKeyInMatch,isFraud,transactionKey
0,358749735,358749735,15000.0,9992.97,19.70,KFC #42585,US,US,5.0,1.0,fastfood,09/2032,2014-12-31,2014-12-31,971,971,8366,PURCHASE,5007.03,True,False,False,T696646
1,699239145,699239145,15000.0,3935.64,193.96,Uber,US,US,9.0,1.0,rideshare,04/2020,2013-12-05,2013-12-05,351,351,1688,PURCHASE,11064.36,False,False,False,T514646
2,854042767,854042767,1000.0,111.98,130.56,Krispy Kreme #205879,US,US,2.0,1.0,fastfood,02/2029,2015-11-13,2016-11-19,554,554,5358,PURCHASE,888.02,True,False,False,T80703
3,770035360,770035360,2500.0,1793.41,46.06,sears.com,US,US,2.0,1.0,online_retail,12/2021,2014-07-08,2014-07-08,946,946,3166,PURCHASE,706.59,False,False,False,T687053
4,894938833,894938833,15000.0,4172.20,286.55,NYSC #160820,US,US,9.0,1.0,health,10/2024,2014-11-05,2016-10-04,689,689,9102,PURCHASE,10827.80,False,False,False,T215349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
785680,207667444,207667444,7500.0,5517.88,385.41,staples.com,US,US,90.0,1.0,online_retail,10/2021,2011-12-12,2011-12-12,235,235,9853,PURCHASE,1982.12,False,False,True,T785680
785713,207667444,207667444,7500.0,764.48,393.43,Pizza Hut #288835,US,US,9.0,8.0,fastfood,10/2027,2011-12-12,2011-12-12,235,235,9853,PURCHASE,6735.52,True,False,True,T785713
785888,428856030,428856030,250.0,61.94,370.43,discount.com,US,US,9.0,1.0,online_retail,08/2025,2009-08-17,2009-08-17,990,990,7660,PURCHASE,188.06,False,False,True,T785888
786054,657364505,657364505,20000.0,20000.00,28.09,Next Day Online Services,US,US,9.0,8.0,online_gifts,03/2029,2014-10-31,2014-10-31,138,138,6163,PURCHASE,0.00,False,False,True,T786054


In [59]:
# Match CVV 
df_tr['CVVInMatch']=np.where( credit_card['cardCVV'] == credit_card['enteredCVV'] , 'True', 'False')


In [60]:
# Drop some variables which are currenly unnecessary in my case
df_tr.drop(columns=['accountNumber','currentExpDate','accountOpenDate','dateOfLastAddressChange','cardLast4Digits'],inplace=True)
df_tr

,customerId,creditLimit,availableMoney,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,posConditionCode,merchantCategoryCode,cardCVV,enteredCVV,transactionType,currentBalance,cardPresent,expirationDateKeyInMatch,isFraud,transactionKey,CVVInMatch
0,358749735,15000.0,9992.97,19.70,KFC #42585,US,US,5.0,1.0,fastfood,971,971,PURCHASE,5007.03,True,False,False,T696646,True
1,699239145,15000.0,3935.64,193.96,Uber,US,US,9.0,1.0,rideshare,351,351,PURCHASE,11064.36,False,False,False,T514646,True
2,854042767,1000.0,111.98,130.56,Krispy Kreme #205879,US,US,2.0,1.0,fastfood,554,554,PURCHASE,888.02,True,False,False,T80703,True
3,770035360,2500.0,1793.41,46.06,sears.com,US,US,2.0,1.0,online_retail,946,946,PURCHASE,706.59,False,False,False,T687053,True
4,894938833,15000.0,4172.20,286.55,NYSC #160820,US,US,9.0,1.0,health,689,689,PURCHASE,10827.80,False,False,False,T215349,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
785680,207667444,7500.0,5517.88,385.41,staples.com,US,US,90.0,1.0,online_retail,235,235,PURCHASE,1982.12,False,False,True,T785680,True
785713,207667444,7500.0,764.48,393.43,Pizza Hut #288835,US,US,9.0,8.0,fastfood,235,235,PURCHASE,6735.52,True,False,True,T785713,True
785888,428856030,250.0,61.94,370.43,discount.com,US,US,9.0,1.0,online_retail,990,990,PURCHASE,188.06,False,False,True,T785888,True
786054,657364505,20000.0,20000.00,28.09,Next Day Online Services,US,US,9.0,8.0,online_gifts,138,138,PURCHASE,0.00,False,False,True,T786054,True


In [61]:
#merchant variables
#merchantCategoryCode
category_group=pd.DataFrame(df_tr.groupby(['merchantCategoryCode','isFraud']).count()['transactionKey']).reset_index()
cf=category_group[category_group.isFraud==False][['merchantCategoryCode','transactionKey']]
cf.columns=['merchantCategoryCode','True_Count']
ct=pd.DataFrame(category_group.groupby(['merchantCategoryCode']).sum()['transactionKey']).reset_index()
ct.columns=['merchantCategoryCode','Total']
cf=cf.merge(ct,on='merchantCategoryCode',how='inner')
cf['Percent']=cf.True_Count/cf.Total
cf=cf.sort_values(by=['Percent'],ascending=True)
cf

,merchantCategoryCode,True_Count,Total,Percent
0,airline,395,825,0.478788
14,online_retail,4448,8659,0.513685
13,online_gifts,869,1648,0.527306
17,rideshare,1143,2140,0.534112
1,auto,475,728,0.652473
5,food,1950,2910,0.670103
8,furniture,201,293,0.686007
3,entertainment,2374,3254,0.729564
18,subscriptions,657,860,0.763953
4,fastfood,3404,4420,0.770136


In [62]:
def categ_merchant(x):
    if x in ['gym','fuel','hotels','online_subscriptions','cable/phone','mobileapps','food_delivery']:
        return 0
    elif x in ['personal care','health','hotels','fastfood','subscriptions','entertainment']:
        return 1
    elif x in ['furniture','food','auto','online_gifts','online_retails','rideshare','arline']:
        return 2

In [63]:
dummies_cm=pd.DataFrame(df_tr.merchantCategoryCode.apply(lambda x: categ_merchant(x)))
dummies_cm.columns=['Categ_merchant']
dummies_cm.Categ_merchant.value_counts()

1.0    9882
2.0    7719
0.0    4321
Name: Categ_merchant, dtype: int64

In [64]:
#merchantCountryCode
country_group=pd.DataFrame(df_tr.groupby(['merchantCountryCode','isFraud']).count()['transactionKey']).reset_index()
c_f=country_group[country_group.isFraud==False][['merchantCountryCode','transactionKey']]
c_f.columns=['merchantCountryCode','True_Count']
c_t=pd.DataFrame(country_group.groupby(['merchantCountryCode']).sum()['transactionKey']).reset_index()
c_t.columns=['merchantCountryCode','Total']
c_f=c_f.merge(c_t,on='merchantCountryCode',how='inner')
c_f['Percent']=c_f.True_Count/c_f.Total
c_f=c_f.sort_values(by=['Percent'],ascending=True)
c_f

,merchantCountryCode,True_Count,Total,Percent
1,MEX,71,120,0.591667
0,CAN,77,121,0.636364
2,PR,40,62,0.645161
3,US,20984,31015,0.676576


In [65]:
def categ_country(x):
    if x in ['US']:
        return 0
    elif x in ['PR','CAN']:
        return 1
    elif x in ['MEX']:
        return 2

In [66]:
dummies_cc=pd.DataFrame(df_tr.merchantCountryCode.apply(lambda x: categ_country(x)))
dummies_cc.columns=['Categ_country']

In [67]:
df_data=pd.concat([dummies_cc,dummies_cm,df_tr],axis=1)
col =df_data.pop("CVVInMatch")
df_data.insert(5, 'CVVinMatch', col)
df_data.replace({False: 0, True: 1}, inplace=True)
df_data

,Categ_country,Categ_merchant,customerId,creditLimit,availableMoney,CVVinMatch,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,posConditionCode,merchantCategoryCode,cardCVV,enteredCVV,transactionType,currentBalance,cardPresent,expirationDateKeyInMatch,isFraud,transactionKey
0,0.0,1.0,358749735,15000.0,9992.97,True,19.70,KFC #42585,US,US,5.0,1.0,fastfood,971,971,PURCHASE,5007.03,1,0,0,T696646
1,0.0,2.0,699239145,15000.0,3935.64,True,193.96,Uber,US,US,9.0,1.0,rideshare,351,351,PURCHASE,11064.36,0,0,0,T514646
2,0.0,1.0,854042767,1000.0,111.98,True,130.56,Krispy Kreme #205879,US,US,2.0,1.0,fastfood,554,554,PURCHASE,888.02,1,0,0,T80703
3,0.0,NaN,770035360,2500.0,1793.41,True,46.06,sears.com,US,US,2.0,1.0,online_retail,946,946,PURCHASE,706.59,0,0,0,T687053
4,0.0,1.0,894938833,15000.0,4172.20,True,286.55,NYSC #160820,US,US,9.0,1.0,health,689,689,PURCHASE,10827.80,0,0,0,T215349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
785680,0.0,NaN,207667444,7500.0,5517.88,True,385.41,staples.com,US,US,90.0,1.0,online_retail,235,235,PURCHASE,1982.12,0,0,1,T785680
785713,0.0,1.0,207667444,7500.0,764.48,True,393.43,Pizza Hut #288835,US,US,9.0,8.0,fastfood,235,235,PURCHASE,6735.52,1,0,1,T785713
785888,0.0,NaN,428856030,250.0,61.94,True,370.43,discount.com,US,US,9.0,1.0,online_retail,990,990,PURCHASE,188.06,0,0,1,T785888
786054,0.0,2.0,657364505,20000.0,20000.00,True,28.09,Next Day Online Services,US,US,9.0,8.0,online_gifts,138,138,PURCHASE,0.00,0,0,1,T786054


In [68]:
df_data.drop(columns=['customerId','merchantName','merchantCountryCode','acqCountry','merchantCategoryCode','transactionType','posEntryMode','posConditionCode'],inplace=True)
                                             

In [69]:
df_data.drop(columns=['currentBalance','transactionKey'],inplace=True)
df_data

,Categ_country,Categ_merchant,creditLimit,availableMoney,CVVinMatch,transactionAmount,cardCVV,enteredCVV,cardPresent,expirationDateKeyInMatch,isFraud
0,0.0,1.0,15000.0,9992.97,True,19.70,971,971,1,0,0
1,0.0,2.0,15000.0,3935.64,True,193.96,351,351,0,0,0
2,0.0,1.0,1000.0,111.98,True,130.56,554,554,1,0,0
3,0.0,NaN,2500.0,1793.41,True,46.06,946,946,0,0,0
4,0.0,1.0,15000.0,4172.20,True,286.55,689,689,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
785680,0.0,NaN,7500.0,5517.88,True,385.41,235,235,0,0,1
785713,0.0,1.0,7500.0,764.48,True,393.43,235,235,1,0,1
785888,0.0,NaN,250.0,61.94,True,370.43,990,990,0,0,1
786054,0.0,2.0,20000.0,20000.00,True,28.09,138,138,0,0,1


In [170]:
df_data.to_csv(abspath_curr+'df_data.csv')